# Serialization

Sometimes, you want to close a program and pick up right where you left off. This might mean ensuring that all the objects are in the state that you want them to be with no further processing. This process of creating a file that will represent the state of some values is called __serialization__. We 'serialize' variables or data structures. Now, Python being Python, they had to give it a more friendly name - pickling.  

One useful approach with pickling is when you are processing text on a server and you are doing something complicated, you can pickle all your current state of things if the program goes sour, then pick up where you left off. The nice thing is that you can put all your variables, objects, etc. in a collection and pickle the collection without worrying about the shape of it. You can only serialize one object at a time, but of course that object can be a collection of numerable other objects. 

Since these files are meant for the computer, they might not make complete sense read as text. But we can save them and read them into a new variable later. This is done with the following syntax: 

~~~ python
import pickle 
x = <object> 
pickle.dump(x,open(<file>,'wb')) 
~~~

And to load the object again (with any name):
~~~ python
y = pickle.load(open(<file>), 'rb')
~~~

In [1]:
import pickle

x = ['1','2']
pickle.dump(x,open("temp.txt",'wb'))
y = pickle.load(open("temp.txt",'rb'))
print(y)

['1', '2']


## Example use case of pickling 

A while back I was processing some Twitter data. One thing I wanted to do was to build egonetworks of users. This would be a social network of a user, the user's followers and crucially the connections between these followers. The algorithm for doing this is a bit tricky. But the important part was that it was slow. You had to more or less look through all the followers of each of an account's followers. This is ok if you look through my few thousand followers or those like many of my colleauges. But every now and then someone would follow Obama or Katy Perry and suddenly you have to download millions of followers. Times like this a program can go sour. So when it does, I often wanted to pickle the state of the progress so I could start up where I left off, rather than start at zero.  

To make a program for collecting data more robust, I did the following: 
1. Catch the ConnectionError event 
2. call a custom pickleProgress() method that dumped all the progress inside a file.
3. The file would be timestamped for the file name.
4. I would retry the connection and if that didn't work the program would shut down. 
5. When restarting the program, I would check if there was a progressPickle file and I would take the most recent one and pick up where I left off.

## Pickles can expire: Check the version number

Notice that we are using ```'rb'``` and ```'wb'``` with the pickles. This is because Python 3's default pickling version writes the pickled object as a bytestream rather than as a series of characters like the text you're reading here. We will discuss bytestreams more in the next chapter when we look at character encodings. In short, bytestreams are strings of characters as readable by a computer, not encoded as text for humans. Between Python 2 and 3, pickles went from being regular characters to bytes, so you cannot read a Python 3 pickle using the previous Python.  

Because pickles are so tightly coupled to the specific version of Python (and the libraries installed even), they are really handy for short term storage but too fragile for long term storage. Instead, one should use one of the file formats discussed above, such as CSV / XML / JSON or even Excel which has extensive support and care with backwards compatibility. If you find yourself with really demanding storage needs, you will probably want to seek out extra resources on this. One example would be to look into the feather package. It was co-written by the creator of Pandas, Wes McKinney, is very fast, compact, and scalable. But it also suffers from the same versioning issues as pickles. Regardless of file type you choose, remember to check both writing the file to disk and re-reading it again before you put it away for a while.  